In [ ]:
"""
This script can be used after manually importing ecoinvent in Activity_Browser.
Look for TODO to define your own variable names as you need.

Author: Ao Chen. Place: UZH Zentrum. 
"""

In [8]:
import os
import pandas as pd
from brightway2 import *
from datetime import date

In [9]:
def project_setup(Project_name):
    projects.set_current(Project_name) 

In [10]:
Project_name = "Test"
Database_name = "cutoff38" 
project_setup(Project_name)

# TODO: change names above as you need

In [14]:
def lca_calculation(Process_list, Material_list_path, Impact_method, Result_path):
    material_df = pd.read_excel(Material_list_path, header=None)
    material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
    material_df['Activity'] = material_df['Activity'].str.strip()

    results = {}

    if not os.path.exists(Result_path):
        os.makedirs(Result_path)

    for i in range(len(material_df)):
    # for i in range(1): # for test
        target_process_list = [p for p in Process_list if material_df['Activity'][i] in str(p)] # search for processes with specific name
        # target_process_list = [p for p in Process_list if 'boron carbide production' in str(p)] # for test
        for j in range(len(target_process_list)):
            functional_unit = {target_process_list[j]: 1}
            lca = LCA(functional_unit, Impact_method)
            lca.lci()
            lca.lcia()
            results[dict(target_process_list[j])['location']] = lca.score
        results_Series = pd.Series(results, name=Result_path+'[kg CO2-Eq]')
        results_Series.to_csv(Result_path+'/Summary_'+material_df['Activity'][i]+'.csv')
        # results_Series.to_csv(Result_path+'/Summary_boron carbide production.csv') # for test

In [15]:
db = Database(Database_name)
Process_list = db.search("*", limit=10000000) # read all processes in the database
# TODO: this process is time consuming, run once is enough!

In [16]:
Material_list_path = 'List_of_Materials_ecoinvent.xlsx'
Impact_method = [m for m in methods if 'IPCC 2013' in str(m) and 'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]
Today = date.today().strftime("%b_%m_%Y")
Result_path = Today+'_calc_results_GWP100a'
lca_calculation(Process_list, Material_list_path, Impact_method, Result_path)
# TODO: change names above as you need